In [1]:
import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

#import plotly
#import plotly.graph_objs as go
#import plotly.offline as offline
#offline.init_notebook_mode(connected=True)

from mpl_toolkits.axes_grid1 import make_axes_locatable
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)

# For changes in .py
%reload_ext autoreload
%autoreload 2

import numpy as np



import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', -1)
pd.set_option('display.max_colwidth', 100)


import pysal as ps

# from pysal.esda.mapclassify import Quantiles, Equal_Interval, Fisher_Jenks


import warnings
warnings.filterwarnings('ignore')


import os
import csv

Populating the interactive namespace from numpy and matplotlib


### Ocorrencias

In [4]:
df_ocorr = pd.read_csv('../dados/ssp_ocorrecencias_registradas.csv')

# mask = df_ocorr['1']!='municipio'
# df_ocorr[mask].shape

In [5]:
mask = df_ocorr.isnull().mean()==1

drop_columns = df_ocorr.isnull().mean()[mask].index.tolist()

df_ocorr = df_ocorr.drop(columns=drop_columns)

In [6]:
mask = (df_ocorr['26'].notnull()) & (df_ocorr['27'].isnull()) & (df_ocorr['28'].isnull()) & (df_ocorr['29'].isnull())
g1 = df_ocorr[mask]

mask = (df_ocorr['27'].notnull()) & (df_ocorr['28'].isnull()) & (df_ocorr['29'].isnull())
g2 = df_ocorr[mask]

mask = (df_ocorr['29'].notnull())
g3 = df_ocorr[mask]

g4 = df_ocorr.drop(index=g1.index)
g4 = g4.drop(index=g2.index)
g4 = g4.drop(index=g3.index)

KeyError: '26'

In [6]:
drop_cols = ['27','28','29']
g1 = g1.drop(columns=drop_cols).reset_index(drop=True)
g1.columns = g1.loc[0].tolist()
mask = g1['municipio']!='municipio'
g1 = g1[mask]


drop_cols = ['28','29']
g2 = g2.drop(columns=drop_cols).reset_index(drop=True)
g2.columns = g2.loc[0].tolist()
mask = g2['municipio']!='municipio'
g2 = g2[mask]

g3 = g3.reset_index(drop=True)
g3.columns = g3.loc[0].tolist()
mask = g3['municipio']!='municipio'
g3 = g3[mask]


drop_cols = ['26','27','28','29']
g4 = g4.drop(columns=drop_cols).reset_index(drop=True)
g4.columns = g4.loc[0].tolist()
mask = g4['municipio']!='municipio'
g4 = g4[mask]


In [7]:
coll_order = g3.columns.tolist()
df = pd.concat([g3,g2,g1,g4], axis=0, sort=False)

df = df[coll_order]

In [8]:
numeric_columns = ['ano',
       'HOMICÍDIO DOLOSO (2)', 'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3)',
       'HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO',
       'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO',
       'HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO',
       'HOMICÍDIO CULPOSO OUTROS', 'TENTATIVA DE HOMICÍDIO',
       'LESÃO CORPORAL SEGUIDA DE MORTE', 'LESÃO CORPORAL DOLOSA',
       'LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO',
       'LESÃO CORPORAL CULPOSA - OUTRAS', 'LATROCÍNIO',
       'Nº DE VÍTIMAS EM LATROCÍNIO', 'TOTAL DE ESTUPRO (4)', 'ESTUPRO',
       'ESTUPRO DE VULNERÁVEL', 'TOTAL DE ROUBO - OUTROS (1)',
       'ROUBO - OUTROS', 'ROUBO DE VEÍCULO', 'ROUBO A BANCO', 'ROUBO DE CARGA',
       'FURTO - OUTROS', 'FURTO DE VEÍCULO']

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [12]:
# df.to_csv('../dados/ssp_ocorrecencias_registradas.csv', index=False, encoding='utf-8')

### Atividades

In [8]:
df_atividades = pd.read_csv('../dados/ssp_atividade_policial.csv')

In [4]:
mask = df_atividades.isnull().mean()==1

drop_columns = df_atividades.isnull().mean()[mask].index.tolist()

df_atividades = df_atividades.drop(columns=drop_columns)

In [5]:
df_atividades.columns = df_atividades.loc[0].tolist()
mask = df_atividades['municipio']!='municipio'
df_atividades = df_atividades[mask]

In [6]:
numeric_columns = [
       'ano',
       'OCORRÊNCIAS DE PORTE DE ENTORPECENTES',
       'OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES',
       'OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1)',
       'OCORRÊNCIAS DE PORTE ILEGAL DE ARMA',
       'Nº DE ARMAS DE FOGO APREENDIDAS',
       'Nº DE FLAGRANTES LAVRADOS',
       'Nº DE INFRATORES APREENDIDOS EM FLAGRANTE',
       'Nº DE INFRATORES APREENDIDOS POR MANDADO',
       'Nº DE PESSOAS PRESAS EM FLAGRANTE',
       'Nº DE PESSOAS PRESAS POR MANDADO',
       'Nº DE PRISÕES EFETUADAS',
       'Nº DE VEÍCULOS RECUPERADOS',
       'TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS'
]

for col in numeric_columns:
    df_atividades[col] = pd.to_numeric(df_atividades[col], errors='coerce')

In [8]:
# df_atividades.to_csv('../dados/ssp_atividade_policial.csv', index=False, encoding='utf-8')